# Input Drift Experiment Outline

*Needs to be double checked*

**Goal**: To identify drift from streams of unseen chest x-ray images

**Mehods:**

1. **Statistical drift detection:** Distance-based measure like Maximum Mean Discrepancy (MMD) to determine the separation between training (reference) data and unseen chest x-rays: 
   - For image data, we will reduce the dimensionality before running the statistical test. 
   - We then run standard CheXpert pre-processing steps and train a drift detector. 
   - We detect data drift by predicting on a batch of x-ray images (spread out over a pre-defined period of time). 
   - We return the **p-value and the threshold** of the test that results in a drift declaration.


2. **Artificial Neural Network (ANN) based drift detection:** Train an autoencoder to learn how to efficiently compress and encode reference data:
   - The AE detector tries to reconstruct the input it receives.
   -  If the unseen, input x-ray cannot be reconstructed well, the reconstruction error is high and the data can be flagged as an outlier (drift).
   - The reconstruction error is measured as the mean squared error (MSE) between the input and the reconstructed instance.


**Requirements:**

- CheXpert training data (reference data)
- Padchest filtered/curated data (new data to be probed for drift)
- Alibi Detect Python library (package with boilerplate code to facilitate methods)

In [1]:
from pathlib import Path

import azureml
from IPython.display import display, Markdown
from azureml.core import Datastore, Experiment, ScriptRunConfig, Workspace, RunConfiguration
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.exceptions import UserErrorException
import shutil


from model_drift import settings, helpers

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.35.0


In [2]:
# Connect to workspace
ws = Workspace.from_config(settings.AZUREML_CONFIG)

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [9]:
dbg = False

log_refresh_rate = 25
if dbg:
    log_refresh_rate = 1

# Name experiement
dataset_name = "chexpert"
dataset_name = "padchest"
env_name = "vae"
experiment_name = f'vae-{dataset_name}' if not dbg else f'vae-{dataset_name}-dbg'


# Input Dataset
dataset = Dataset.get_by_name(ws, name=dataset_name)

# Experiment
exp = Experiment(workspace=ws, name=experiment_name)

# Environment
environment_file = settings.CONDA_ENVIRONMENT_FILE
project_dir = settings.SRC_DIR
pytorch_env = Environment.from_conda_specification(env_name, file_path=str(environment_file))
pytorch_env.register(workspace=ws)
build = pytorch_env.build(workspace=ws)

# Run Configuration
run_config = RunConfiguration()
env_vars = {
    "AZUREML_COMPUTE_USE_COMMON_RUNTIME": "false",
    "RSLEX_DIRECT_VOLUME_MOUNT": "true"
}
for k, v in env_vars.items():
    pytorch_env.environment_variables[k] = v
    run_config.environment_variables[k] = v
run_config.environment = pytorch_env
run_config.docker = DockerConfiguration(use_docker=True, shm_size="100G")


args = {
    'dataset': dataset_name,
    'run_azure': 1,
    'output_dir': './outputs',

    'frontal_only': 1,
    'val_frontal_only': 0,
    'ignore_nonfrontal_loss': 1,

    'batch_size': 32,
    'base_lr': 0.0001,
    'image_size': 128,

    'max_epochs': 50 if not dbg else 5,
    'num_workers': -1,

    'progress_bar_refresh_rate': log_refresh_rate,
    'log_every_n_steps': log_refresh_rate,
    'flush_logs_every_n_steps': log_refresh_rate,

    'accelerator': 'ddp',
    'channels': 1,
    'normalize': False,

    'step_size': 3,
    'lr_scheduler': 'plateau',
    'auto_scale_batch_size': False,
    'auto_lr_find': False,

    'width': 320,
    'z': 64,
    'layer_count': 3,
    'terminate_on_nan': True,
    'log_recon_images': 32
}

if dbg:
    args.update({
        'limit_train_batches': 5,
        'limit_val_batches': 5,
        'num_sanity_val_steps': 5
    })


args['data_folder'] = Dataset.get_by_name(ws, name=args['dataset']).as_named_input('dataset').as_mount()

print("args:")
for k, v in sorted(args.items()):
    print(f" {k}: {v}")


print(f"Environment: {pytorch_env.name}")
print(f"Experiment: {exp.name}")

config = ScriptRunConfig(
    source_directory=str(project_dir),
    script="scripts/vae/train.py",
    arguments=helpers.argsdict2list(args),
)
config.run_config = run_config


args:
 accelerator: ddp
 auto_lr_find: False
 auto_scale_batch_size: False
 base_lr: 0.0001
 batch_size: 32
 channels: 1
 data_folder: <azureml.data.dataset_consumption_config.DatasetConsumptionConfig object at 0x000001476F005710>
 dataset: padchest
 flush_logs_every_n_steps: 25
 frontal_only: 1
 ignore_nonfrontal_loss: 1
 image_size: 128
 layer_count: 3
 log_every_n_steps: 25
 log_recon_images: 32
 lr_scheduler: plateau
 max_epochs: 50
 normalize: False
 num_workers: -1
 output_dir: ./outputs
 progress_bar_refresh_rate: 25
 run_azure: 1
 step_size: 3
 terminate_on_nan: True
 val_frontal_only: 0
 width: 320
 z: 64
Environment: vae
Experiment: vae-padchest


In [8]:

config.run_config.target = "nc24-uswest2"
config.run_config.target = "NC24rs-v3-usw2-d"

run = exp.submit(config)
display(Markdown(f"""
- Environment: {pytorch_env.name}
- Experiment: [{run.experiment.name}]({run.experiment.get_portal_url()})
- Run: [{run.display_name}]({run.get_portal_url()})
- Target: {config.run_config.target}
"""))


- Environment: vae
- Experiment: [vae-padchest-dbg](https://ml.azure.com/experiments/vae-padchest-dbg?wsid=/subscriptions/9ca8df1a-bf40-49c6-a13f-66b72a85f43c/resourcegroups/MLOps-Prototype/workspaces/MLOps_shared&tid=72f988bf-86f1-41af-91ab-2d7cd011db47)
- Run: [witty_stamp_g1xnkm5m](https://ml.azure.com/runs/vae-padchest-dbg_1637349922_316117da?wsid=/subscriptions/9ca8df1a-bf40-49c6-a13f-66b72a85f43c/resourcegroups/MLOps-Prototype/workspaces/MLOps_shared&tid=72f988bf-86f1-41af-91ab-2d7cd011db47)
- Target: NC24rs-v3-usw2-d


## Explain hyper drive
TODO

In [16]:
args["max_epochs"] = 100

In [20]:
from azureml.train.hyperdrive import HyperDriveRun
from azureml.train.hyperdrive import GridParameterSampling, RandomParameterSampling, BanditPolicy, HyperDriveConfig, uniform, PrimaryMetricGoal, choice, loguniform
run_config = RunConfiguration()

cluster_name = "nc24-uswest2"


parents = ["HD_135c0441-47f5-4414-bc37-ab89758a7955"]

parents = [HyperDriveRun(exp, p) for p in parents]


run_config.environment = pytorch_env
run_config.docker = DockerConfiguration(use_docker=True, shm_size="100G")
run_config.target = cluster_name


param_sampling = RandomParameterSampling(
    {
        "layer_count": choice(4),
        "batch_size": choice(16, 32),
        "image_size": choice(128),
        "z": choice(128),
        "width": choice(160, 240, 320),
        "base_lr": choice(1e-4, 1e-5),
        "kl_coeff": choice(0.1)
    }
)

experiment_name = f'vae-{dataset_name}-tune'
exp = Experiment(workspace=ws, name=experiment_name)
config.run_config = run_config

from azureml.train.hyperdrive import BanditPolicy
early_termination_policy = BanditPolicy(slack_factor = 0.5, evaluation_interval=1, delay_evaluation=25)

hyperdrive_config = HyperDriveConfig(run_config=config,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     resume_from=parents,
                                     primary_metric_name='val/weighted_recon_loss',
                                     primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                                     max_total_runs=6*12,
                                     max_concurrent_runs=4)


print("args:")
for k,v in sorted(zip(config.arguments[::2], config.arguments[1::2])):
    k = k.strip("-")
    v = param_sampling._parameter_space.get(k, v)
    print(f" {k}: {v}")

print(f"Environment: {pytorch_env.name}")
print(f"Experiment: {exp.name}")

args:
 accelerator: ddp
 auto_lr_find: False
 auto_scale_batch_size: False
 base_lr: ['choice', [[0.0001, 1e-05]]]
 batch_size: ['choice', [[16, 32]]]
 channels: 1
 data_folder: <azureml.data.dataset_consumption_config.DatasetConsumptionConfig object at 0x000001476F005710>
 dataset: padchest
 flush_logs_every_n_steps: 25
 frontal_only: 1
 ignore_nonfrontal_loss: 1
 image_size: ['choice', [[128]]]
 layer_count: ['choice', [[4]]]
 log_every_n_steps: 25
 log_recon_images: 32
 lr_scheduler: plateau
 max_epochs: 50
 normalize: False
 num_workers: -1
 output_dir: ./outputs
 progress_bar_refresh_rate: 25
 run_azure: 1
 step_size: 3
 terminate_on_nan: True
 val_frontal_only: 0
 width: ['choice', [[160, 240, 320]]]
 z: ['choice', [[128]]]
Environment: vae
Experiment: vae-padchest-tune


In [21]:
# start the HyperDrive run
hyperdrive_run = exp.submit(hyperdrive_config)
display(Markdown(f"""
- Experiement: [{hyperdrive_run.experiment.name}]({hyperdrive_run.experiment.get_portal_url()})
- Run: [{hyperdrive_run.display_name}]({hyperdrive_run.get_portal_url()})
- Target: {config.run_config.target}
"""))


- Experiement: [vae-padchest-tune](https://ml.azure.com/experiments/vae-padchest-tune?wsid=/subscriptions/9ca8df1a-bf40-49c6-a13f-66b72a85f43c/resourcegroups/MLOps-Prototype/workspaces/MLOps_shared&tid=72f988bf-86f1-41af-91ab-2d7cd011db47)
- Run: [neat_rainbow_pc3vdg7b](https://ml.azure.com/runs/HD_e52267ff-4ea9-441b-aacc-27e6316168ee?wsid=/subscriptions/9ca8df1a-bf40-49c6-a13f-66b72a85f43c/resourcegroups/MLOps-Prototype/workspaces/MLOps_shared&tid=72f988bf-86f1-41af-91ab-2d7cd011db47)
- Target: nc24-uswest2
